In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class MotionModel:
    def __init__(self, position, friction=0.7):
        assert 0 <= friction <= 1

        self.position = position.copy()
        self.velocity = np.array([0, 0])
        self.friction = friction

    def update(self, position):
        self.velocity = (self.friction * self.velocity +
                         (1 - self.friction) * (position - self.position))
        self.position += self.velocity

def rand_position_step(a=-1, b=2):
    return np.random.random(2) * (b - a) + a

position = np.array([0, 0], dtype=np.float)
model = MotionModel(position)
n_iterations = 10
iterations = np.arange(0, n_iterations)

positions_real = []
positions_model = []

for i in iterations:
    positions_real.append(position.copy())
    positions_model.append(model.position.copy())
    step = rand_position_step()
    position += step
    model.update(position)

positions_real = np.array(positions_real)
positions_model = np.array(positions_model)

plt.scatter(positions_real[:, 0], positions_real[:, 1], s=10, label='real')
plt.scatter(positions_model[:, 0], positions_model[:, 1], s=10, label='model')
plt.legend()
plt.show()